## Step 1: Environment Setup

In [1]:
import os
import subprocess
import sys
import time
import pandas as pd
import json
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

In [2]:
REGION = 'us-central1'
#ENDPOINT = '<YOUR_ENDPOINT>' e.g. '337dd39580cbcbd2-dot-us-central2.pipelines.googleusercontent.com'
INPUT_FILE = 'gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx'
ARTIFACT_STORE_URI = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [3]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'X_train.csv')
TRAINING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'training')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'X_validate.csv')
VALIDATION_FILE_DIR='{}/{}'.format(DATA_ROOT, 'validation')
TESTING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'testing')
TESTING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'testing', 'X_test.csv')

## Step 2a: Write the training APP

In [4]:
TRAIN_IMAGE_FOLDER = 'train_image'
os.makedirs(TRAIN_IMAGE_FOLDER, exist_ok=True)

## Step 2b: Write the training script

In [5]:
%%writefile {TRAIN_IMAGE_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import json

import pickle
import time


import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split  ## from analysis

#simple sklearn impute and scale numeric pipeline
from sklearn.pipeline import Pipeline ## from analysis
from sklearn.impute import SimpleImputer ## from analysis
from sklearn.preprocessing import StandardScaler ## from analysis
import numpy as np ## from analysis

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV


import functools


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from googleapiclient import discovery
from googleapiclient import errors




def train_evaluate(job_dir, input_file, training_dataset, validation_dataset, testing_dataset, n_estimators, max_depth, min_samples_leaf, max_features, min_samples_split, class_weight, max_leaf_nodes, random_state, hptune, bootstrap):

   
    obj = input_file
    print("obj", obj)
    data = pd.read_excel(obj,sheet_name='data') 
    meta_data = pd.read_excel(obj,sheet_name='meta data') 
    
    ## Preprocess    
    #Prepare data for analysis
    #Split out numeric from categorical varibles

    ##var_type_filter = [x in ['physiological','biochemical','process'] for x in meta_data['variable type']]
    var_type_filter = [x in ['independent'] for x in meta_data['variable type']]
    var_dtype_filter = (data.dtypes == 'float64') | (data.dtypes == 'int64')

    numeric_vars = (var_type_filter & var_dtype_filter).values
    numeric_x_data = data[data.columns[numeric_vars]]

    #things to try to predict
    y_data = data[data.columns[(meta_data['target'] == 1).values]]

    #meta data about variables
    meta_data = meta_data.query('name in {}'.format(list(data.columns[numeric_vars].values))).set_index('name')
 
    
    
    #Variables which will be used to build the model
    model_target = 'Run_Performance' ## Select target for classification
    
    y_data = data[[model_target]]
    
    
    #maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(numeric_x_data, y_data, test_size=0.25, stratify = y_data[model_target], random_state=42)

    #split train set to create a pseudo test or validation dataset
    X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.33, stratify= y_train[model_target], random_state=42)
    
    
    print('The training, validation and test data contain {}, {} and {} rows respectively'.format(len(X_train),len(X_validate),len(X_test)))

    training_file = 'X_train'
    testing_file = 'X_test'
    validation_file = 'X_validate'
    serving_file = 'X_serving'
  
    Xy_train = X_train.join(y_train)
    cmd = "Xy_train.to_csv('{}/{}.csv', index=False)".format(training_dataset, training_file)
    eval(cmd)
    print("Saved training files for later..")
    Xy_test =X_test.join(y_test)
    cmd = "Xy_test.to_csv('{}/{}.csv', index=False)".format(testing_dataset, testing_file)
    print("Saved testing files for later..")
    eval(cmd)
  
    cmd = "X_test.to_csv('{}/{}.csv', index=False)".format(testing_dataset, serving_file)
    print("Saved serving instance files for later..")
    eval(cmd)

    Xy_validate =X_validate.join(y_validate)
    cmd = "X_validate.to_csv('{}/{}.csv', index=False)".format(validation_dataset, validation_file)
    eval(cmd)
    print("Saved validation files for later..")

    
    if not hptune:
        #df_train = pd.concat([df_train, df_validation])
        X_train = pd.concat([X_train, X_validate])
        y_train = pd.concat([y_train, y_validate])
        


    ## Train, optimize and validate predictive model
    ### Train



    classifier = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf,
                max_features=max_features,
                min_samples_split=min_samples_split,
                class_weight=class_weight,
                max_leaf_nodes=max_leaf_nodes,
                random_state=random_state,
                bootstrap=bootstrap

     )

    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    #auto scale
    scaler = StandardScaler()

    estimator = Pipeline([
      ('imputer', imputer),
      ('scaler', scaler),
      ('classifier', classifier),
    ])
    

    #prepare data for modeling
    #use the pipeline created above

    #_X_train = pipe.fit_transform(X_train)
    _X_train = X_train    
    _y_train = y_train[model_target]    ## selected target label for prediction
    _X_validate = X_validate
    _y_validate = y_validate[model_target]

    _X_test = X_test
    _y_test = y_test[model_target]


    
    print('Starting training: n_estimators={}, max_depth={}, min_samples_leaf={}, max_features={}, min_samples_split={}, class_weight={}, max_leaf_nodes={}, random_state={}, hptune={}, bootstrap={}'.format(n_estimators, max_depth, min_samples_leaf, max_features, min_samples_split, class_weight, max_leaf_nodes, random_state, hptune, bootstrap))

    #estimator.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter) 
    estimator.set_params(classifier__n_estimators=n_estimators, classifier__max_depth=max_depth, classifier__min_samples_leaf=min_samples_leaf, 
                         classifier__max_features=max_features, classifier__min_samples_split=min_samples_split, classifier__class_weight=class_weight,
                         classifier__max_leaf_nodes=max_leaf_nodes, classifier__random_state=random_state, classifier__bootstrap=bootstrap) 
    #pipeline.fit(X_train, y_train)
    estimator.fit(_X_train, _y_train)

    
    if hptune:
        accuracy = estimator.score(_X_validate, _y_validate)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(estimator, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 

    

        
if __name__ == "__main__":
    fire.Fire(train_evaluate)





Overwriting train_image/train.py


## Step 3: Package the script into a docker image.

In [112]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


Overwriting training_app/Dockerfile


In [6]:
%%writefile {TRAIN_IMAGE_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing train_image/Dockerfile


## Step 4: Build the docker image

In [7]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)
TRAINER_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [8]:
!gcloud builds submit --tag $TRAINER_IMAGE $TRAIN_IMAGE_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 8.0 KiB before compression.
Uploading tarball of [train_image] to [gs://etl-project-datahub_cloudbuild/source/1599457079.49595-3272fcd80ebf482caa9e97bc0ee56ea7.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/71d38685-cadf-423b-b959-414a93bfcf0d].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/71d38685-cadf-423b-b959-414a93bfcf0d?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "71d38685-cadf-423b-b959-414a93bfcf0d"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599457079.49595-3272fcd80ebf482caa9e97bc0ee56ea7.tgz#1599457079972183
Copying gs://etl-project-datahub_cloudbuild/source/1599457079.49595-3272fcd80ebf482caa9e97bc0ee56ea7.tgz#1599457079972183...
/ [1 files][  2.8 KiB/  2.8 KiB]                                                
Operation completed over 1 o

In [183]:
!gcloud builds submit --tag $TRAINER_IMAGE $TRAINING_APP_FOLDER

Creating temporary tarball archive of 21 file(s) totalling 105.3 KiB before compression.
Uploading tarball of [training_app] to [gs://etl-project-datahub_cloudbuild/source/1599454981.41656-9664c7ae35c44d2c805c3df46f258a55.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/8c3e8385-4c81-4356-b88b-dbcde075aef1].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/8c3e8385-4c81-4356-b88b-dbcde075aef1?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8c3e8385-4c81-4356-b88b-dbcde075aef1"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599454981.41656-9664c7ae35c44d2c805c3df46f258a55.tgz#1599454981875812
Copying gs://etl-project-datahub_cloudbuild/source/1599454981.41656-9664c7ae35c44d2c805c3df46f258a55.tgz#1599454981875812...
/ [1 files][ 26.4 KiB/ 26.4 KiB]                                                
Operation completed over

In [114]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 21 file(s) totalling 105.3 KiB before compression.
Uploading tarball of [training_app] to [gs://etl-project-datahub_cloudbuild/source/1599441013.51649-f0124bcd32aa4f2d8fafc985a9ea39fa.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/d8ce2747-8f95-4e43-a92b-e209fb008e53].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/d8ce2747-8f95-4e43-a92b-e209fb008e53?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "d8ce2747-8f95-4e43-a92b-e209fb008e53"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599441013.51649-f0124bcd32aa4f2d8fafc985a9ea39fa.tgz#1599441013924331
Copying gs://etl-project-datahub_cloudbuild/source/1599441013.51649-f0124bcd32aa4f2d8fafc985a9ea39fa.tgz#1599441013924331...
/ [1 files][ 26.4 KiB/ 26.4 KiB]                                                
Operation completed over

# Step 5: Run hyperparameter tuning jobs

### Create the hyperparameter configuration file.



#### Recall that the training code uses SGDClassifier. The training application has been designed to accept two hyperparameters that control SGDClassifier:

* ##### CLASS_WEIGHT 
* ##### MAX_DEPTH 
* ##### MAX_FEATURES 
* ##### MAX_LEAF_NODES 
* ##### MIN_SAMPLE_LEAF 
* ##### MIN_SAMPLE_SPLIT 
* ##### N_ESTIMATORS 
* ##### RANDOM_STATE

#### The below file configures AI Platform hypertuning to run up to 4 trials on up to three nodes and to choose from three discrete values of min_samples_split and the linear range between 10 and 200 for n_estimators.

In [13]:
%%writefile {TRAIN_IMAGE_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 4
    maxParallelTrials: 4
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE
    algorithm: RANDOM_SEARCH
    params:
    - parameterName: n_estimators
      type: INTEGER
      minValue: 10
      maxValue: 200
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: max_depth
      type: INTEGER
      minValue: 3
      maxValue: 100
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: min_samples_leaf
      type: INTEGER
      minValue: 10
      maxValue: 500
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: max_features
      type: DOUBLE
      minValue: 0.5
      maxValue: 1.0
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: min_samples_split
      type: DISCRETE
      discreteValues: [
          2,
          5,
          10
      ]
    - parameterName: class_weight
      type: CATEGORICAL
      categoricalValues: [
          "balanced",
          "balanced_subsample"
      ]
    - parameterName: max_leaf_nodes
      type: INTEGER
      minValue: 10
      maxValue: 500
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: random_state
      type: INTEGER
      minValue: 35
      maxValue: 75
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: bootstrap
      type: CATEGORICAL
      categoricalValues: [
          "TRUE",
          "FALSE"
      ]

Overwriting train_image/hptuning_config.yaml


## Step 5a: Start the hyperparameter tuning job

job_dir -> GCS Path for storing the job packages & model.

training_dataset_path -> GCS path holding training dataset.

validation_dataset_path -> GCS path holding validation dataset.

alpha -> hyperparameter

max_iter -> hyperparameter

hptune -> variable to decide if hyperparameter tuning is to be done or not.


In [11]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'X_train.xlsx')
TRAINING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'training')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'X_validate.xlsx')
VALIDATION_FILE_DIR='{}/{}'.format(DATA_ROOT, 'validation')
TESTING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'testing')

In [12]:
## Hyperparameter Tuning Job
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"
  

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$TRAINER_IMAGE \
--scale-tier=$SCALE_TIER \
--config $TRAIN_IMAGE_FOLDER/hptuning_config.yaml \
-- \
--training_dataset=$TRAINING_FILE_DIR \
--validation_dataset=$VALIDATION_FILE_DIR \
--testing_dataset=$TESTING_FILE_DIR \
--input_file=$INPUT_FILE \
--hptune

Job [JOB_20200907_054530] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200907_054530

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200907_054530
jobId: JOB_20200907_054530
state: QUEUED


## Step 5b:Monitor the job

In [14]:
#Hyperparameter Tuning Job description
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-09-07T05:45:32Z'
endTime: '2020-09-07T05:53:20Z'
etag: gj2qHf1qyIA=
jobId: JOB_20200907_054530
startTime: '2020-09-07T05:45:34Z'
state: SUCCEEDED
trainingInput:
  args:
  - --training_dataset=gs://workshop_trial_artifact_store_pp/data/training
  - --validation_dataset=gs://workshop_trial_artifact_store_pp/data/validation
  - --testing_dataset=gs://workshop_trial_artifact_store_pp/data/testing
  - --input_file=gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx
  - --hptune
  hyperparameters:
    algorithm: RANDOM_SEARCH
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 4
    maxTrials: 4
    params:
    - maxValue: 200.0
      minValue: 10.0
      parameterName: n_estimators
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 100.0
      minValue: 3.0
      parameterName: max_depth
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 500.0
      minValue:

## Step 5c:Monitor the job - stream the progress

In [86]:
# for streaming the logs
#!gcloud ai-platform jobs stream-logs $JOB_NAME

^C


Command killed by keyboard interrupt



## Step 6c: Retrieve HP-tuning results.

In [15]:
from googleapiclient import discovery
from googleapiclient import errors
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response


{'jobId': 'JOB_20200907_054530',
 'trainingInput': {'args': ['--training_dataset=gs://workshop_trial_artifact_store_pp/data/training',
   '--validation_dataset=gs://workshop_trial_artifact_store_pp/data/validation',
   '--testing_dataset=gs://workshop_trial_artifact_store_pp/data/testing',
   '--input_file=gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'n_estimators',
     'minValue': 10,
     'maxValue': 200,
     'type': 'INTEGER',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'max_depth',
     'minValue': 3,
     'maxValue': 100,
     'type': 'INTEGER',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'min_samples_leaf',
     'minValue': 10,
     'maxValue': 500,
     'type': 'INTEGER',
     'scaleType': 'UNIT_LINEAR_SCALE'},
    {'parameterName': 'max_features',
     'minValue': 0.5,
     'maxValue': 1,
     'type': 'DOUBLE',
     'scaleType': '

In [16]:
response['trainingOutput']['trials'][0]

{'trialId': '1',
 'hyperparameters': {'max_leaf_nodes': '364',
  'min_samples_leaf': '11',
  'max_features': '0.56918819952535549',
  'n_estimators': '127',
  'bootstrap': 'TRUE',
  'class_weight': 'balanced_subsample',
  'max_depth': '18',
  'min_samples_split': '5',
  'random_state': '53'},
 'finalMetric': {'trainingStep': '1', 'objectiveValue': 0.9588688946015425},
 'startTime': '2020-09-07T05:46:11.933947341Z',
 'endTime': '2020-09-07T05:52:42Z',
 'state': 'SUCCEEDED'}

# Step 7: Retrain the model with the best hyperparameters

### Configure and run the training job

In [17]:
# Selecting the best parameters
CLASS_WEIGHT = response['trainingOutput']['trials'][0]['hyperparameters']['class_weight']
MAX_DEPTH = response['trainingOutput']['trials'][0]['hyperparameters']['max_depth']
MAX_FEATURES = response['trainingOutput']['trials'][0]['hyperparameters']['max_features']
MAX_LEAF_NODES = response['trainingOutput']['trials'][0]['hyperparameters']['max_leaf_nodes']
MIN_SAMPLE_LEAF = response['trainingOutput']['trials'][0]['hyperparameters']['min_samples_leaf']
MIN_SAMPLE_SPLIT = response['trainingOutput']['trials'][0]['hyperparameters']['min_samples_split']
N_ESTIMATORS = response['trainingOutput']['trials'][0]['hyperparameters']['n_estimators']
RANDOM_STATE = response['trainingOutput']['trials'][0]['hyperparameters']['random_state']
BOOTSTRAP = response['trainingOutput']['trials'][0]['hyperparameters']['bootstrap']

In [18]:
#Now select the best parameters for deploying the model
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"
#BOOTSTRAP = "TRUE"
#CLASS_WEIGHT = "balanced_subsample"
#MAX_DEPTH = "22"
#MAX_FEATURES = "0.92715679885914515"
#MAX_LEAF_NODES = "156"
#MIN_SAMPLE_LEAF = "55"
#MIN_SAMPLE_SPLIT = "5"
#N_ESTIMATORS = "147"
#RANDOM_STATE = "37"


!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$TRAINER_IMAGE \
--scale-tier=$SCALE_TIER \
-- \
--training_dataset=$TRAINING_FILE_DIR \
--validation_dataset=$VALIDATION_FILE_DIR \
--testing_dataset=$TESTING_FILE_DIR \
--input_file=$INPUT_FILE \
--bootstrap=$BOOTSTRAP \
--class_weight=$CLASS_WEIGHT \
--max_depth=$MAX_DEPTH \
--max_features=$MAX_FEATURES \
--min_samples_leaf=$MIN_SAMPLE_LEAF \
--max_leaf_nodes=$MAX_LEAF_NODES \
--min_samples_split=$MIN_SAMPLE_SPLIT \
--n_estimators=$N_ESTIMATORS \
--random_state=$RANDOM_STATE \
--nohptune

Job [JOB_20200907_055554] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200907_055554

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200907_055554
jobId: JOB_20200907_055554
state: QUEUED


In [19]:
from googleapiclient import discovery
from googleapiclient import errors
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response


{'jobId': 'JOB_20200907_055554',
 'trainingInput': {'args': ['--training_dataset=gs://workshop_trial_artifact_store_pp/data/training',
   '--validation_dataset=gs://workshop_trial_artifact_store_pp/data/validation',
   '--testing_dataset=gs://workshop_trial_artifact_store_pp/data/testing',
   '--input_file=gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx',
   '--bootstrap=TRUE',
   '--class_weight=balanced_subsample',
   '--max_depth=18',
   '--max_features=0.56918819952535549',
   '--min_samples_leaf=11',
   '--max_leaf_nodes=364',
   '--min_samples_split=5',
   '--n_estimators=127',
   '--random_state=53',
   '--nohptune'],
  'region': 'us-central1',
  'jobDir': 'gs://workshop_trial_artifact_store_pp/jobs/JOB_20200907_055554',
  'masterConfig': {'imageUri': 'gcr.io/etl-project-datahub/trainer_image:latest'}},
 'createTime': '2020-09-07T05:55:56Z',
 'startTime': '2020-09-07T05:59:44Z',
 'endTime': '2020-09-07T06:02:16Z',
 'state': 'SUCCEEDED',
 'trainingOutput': {'cons

# Model and Model Version

In [20]:
model_name = 'amyris_RFClassifiervRFC1'
labels = "task=classifier,domain=biotech"
filter = 'name:{}'.format(model_name)
models = !(gcloud ai-platform models list --filter={filter} --format='value(name)')

#if not models:
!gcloud ai-platform models create  $model_name \
    --regions=$REGION \
    --labels=$labels
#else:
#    print("Model: {} already exists.".format(models[0]))

Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/etl-project-datahub/models/amyris_RFClassifiervRFC1].


In [21]:
model_version = 'v01'
filter = 'name:{}'.format(model_version)
# versions = !(gcloud ai-platform versions list --model={model_name} --format='value(name)' --filter={filter})


#if not versions:
!gcloud ai-platform versions create {model_version} \
    --model={model_name} \
    --origin=$JOB_DIR \
    --runtime-version=1.15 \
    --framework=scikit-learn \
    --python-version=3.7
#else:
#     print("Model version: {} already exists.".format(versions[0]))

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


# Now Predict with new data

## input file is in json format. Use that for prediction

In [23]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'X_train.xlsx')
TRAINING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'training')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'X_validate.xlsx')
VALIDATION_FILE_DIR='{}/{}'.format(DATA_ROOT, 'validation')
TESTING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'testing')

In [24]:
TESTING_FILE_DIR

'gs://workshop_trial_artifact_store_pp/data/testing'

## Serve predictions

### Prepare the input file with JSON formated instances.

In [25]:
import json
input_file = 'serving_instance_2a.json'
X_test_file= '{}/{}'.format(TESTING_FILE_DIR,'X_serving.csv')
X_test_file
##'gs://workshop_trial_artifact_store_pp/data/testing/X_test.csv'
##'gs://workshop_trial_artifact_store_pp/data/testing/X_test.csv'

#X_test = pd.read_csv('gs://workshop_trial_artifact_store_pp/data/testing/X_test.csv')
X_test = pd.read_csv(X_test_file)

X_test.head()

with open(input_file, 'w') as f:
    for index, row in X_test.head().iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')


In [26]:
!cat $input_file

[NaN, NaN, NaN, NaN, NaN, NaN, 1.022618, NaN, NaN, 0.580472404447558, 98.58567, 94.81158, 93.470631380586, 4.0, 4.0, 0.0, 0.00169426013012948, 14.95093, 33.61875, NaN, 78.16006999999999, 126.72975, 16.18621, 126.72975, 78.16006999999999, 0.02351, 0.00526, 0.0224465933240588, 140.0, 70.52042, 2.1858920909737303, NaN, NaN, NaN, NaN, NaN, NaN, 100000000.0, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, 0.61466, NaN, 0.5989, 0.0, 31.237190000000002, 14.02679, NaN, 4.279656, 711.2092200000001, 10.7049]
[42.6409517041319, 0.0193221824684571, 0.0118420543402428, 0.00253719182104052, 0.0036820543402428297, 0.7213146226785458, 0.497036, 0.48678195857648104, 2.08721842367735, 0.33542666397717497, 97.62640999999999, 89.44386999999999, 87.32083924606701, 6.0, 6.0, 0.0, 0.0008921167757984731, 69.745, 99.41664, 0.58315, 355.05412, 524.79891, NaN, 524.79891, 355.05412, 0.008159999999999999, 0.00312, 0.013314329119974, 90.0, 89.00098, 1.4682557528184, NaN, NaN, NaN, NaN, NaN, NaN, Na

### Invoke the model

In [27]:
!gcloud ai-platform predict \
--model $model_name \
--version $model_version \
--json-instances $input_file

Using endpoint [https://ml.googleapis.com/]
['delta', 'gamma', 'delta', 'delta', 'delta']


# Kubeflow Pipeline

## Continuous training pipeline with KFP and Cloud AI Platform

In [28]:
TRAINER_IMAGE

'gcr.io/etl-project-datahub/trainer_image:latest'

In [107]:
#!grep 'BASE_IMAGE =' -A 5 pipeline/train.py

grep: pipeline/train.py: No such file or directory


In [29]:
TESTING_FILE_PATH

'gs://workshop_trial_artifact_store_pp/data/testing/X_test.csv'

In [30]:
PIPELINE_FOLDER = 'pipeline'
os.makedirs(PIPELINE_FOLDER, exist_ok=True)

In [32]:
%%writefile ./pipeline/amyris_pipeline.py
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""KFP pipeline orchestrating BigQuery and Cloud AI Platform services."""

import os

from helper_components import evaluate_model
from helper_components import retrieve_best_run
from jinja2 import Template
import kfp
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

# Defaults and environment settings
BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')
USE_KFP_SA = os.getenv('USE_KFP_SA')
TRAINING_FILE_PATH = 'gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx'
INPUT_FILE = 'gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx'
TRAINING_FILE_DIR = 'gs://workshop_trial_artifact_store_pp/data/training'
TESTING_FILE_DIR = 'gs://workshop_trial_artifact_store_pp/data/testing'
VALIDATION_FILE_DIR = 'gs://workshop_trial_artifact_store_pp/data/validation'


TESTING_FILE_PATH = 'gs://workshop_trial_artifact_store_pp/data/testing/X_test.csv'
# VALIDATION_FILE_PATH = 'datasets/validation/data.csv'
# TESTING_FILE_PATH = 'datasets/testing/data.csv'

# Parameter defaults
# SPLITS_DATASET_ID = 'splits'
HYPERTUNE_SETTINGS = """
{
    "hyperparameters":  {
        "goal": "MAXIMIZE",
        "maxTrials": 3,
        "maxParallelTrials": 3,
        "hyperparameterMetricTag": "accuracy",
        "enableTrialEarlyStopping": True,
        "algorithm": "RANDOM_SEARCH",
        "params": [
            {
                "parameterName": "n_estimators",
                "type": "INTEGER",
                "minValue": 10,
                "maxValue": 200,
                "scaleType": "UNIT_LINEAR_SCALE"
            },
            {
                "parameterName": "max_leaf_nodes",
                "type": "INTEGER",
                "minValue": 10,
                "maxValue": 500,
                "scaleType": "UNIT_LINEAR_SCALE"
            },
            {
                "parameterName": "max_depth",
                "type": "INTEGER",
                "minValue": 3,
                "maxValue": 20,
                "scaleType": "UNIT_LINEAR_SCALE"
            },
            {
                "parameterName": "min_samples_split",
                "type": "DISCRETE",
                "discreteValues": [2,5,10]
            },
            {
                "parameterName": "min_samples_leaf",
                "type": "INTEGER",
                "minValue": 10,
                "maxValue": 500,
                "scaleType": "UNIT_LINEAR_SCALE"
            },
            {
                "parameterName": "max_features",
                "type": "DOUBLE",
                "minValue": 0.5,
                "maxValue": 1.0,
                "scaleType": "UNIT_LINEAR_SCALE"
            },    
            {
                "parameterName": "class_weight",
                "type": "CATEGORICAL",
                "categoricalValues": [
                              "balanced",
                              "balanced_subsample"
                          ]
            },  

             {
                "parameterName": "random_state",
                "type": "INTEGER",
                "minValue": 35,
                "maxValue": 75,
                "scaleType": "UNIT_LINEAR_SCALE"
            },
            {
                "parameterName": "bootstrap",
                "type": "CATEGORICAL",
                "categoricalValues": [
                              "TRUE",
                              "FALSE"
                          ]
            } 
        ]
    }
}
"""

# # Helper functions
# def generate_sampling_query(source_table_name, num_lots, lots):
#     """Prepares the data sampling query."""

#     sampling_query_template = """
#          SELECT *
#          FROM 
#              `{{ source_table }}` AS cover
#          WHERE 
#          MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
#          """
#     query = Template(sampling_query_template).render(
#         source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

#     return query


# Create component factories
component_store = kfp.components.ComponentStore(
    local_search_paths=None, url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])

# bigquery_query_op = component_store.load_component('bigquery/query')
mlengine_train_op = component_store.load_component('ml_engine/train')
mlengine_deploy_op = component_store.load_component('ml_engine/deploy')
retrieve_best_run_op = func_to_container_op(
    retrieve_best_run, base_image=BASE_IMAGE)
evaluate_model_op = func_to_container_op(evaluate_model, base_image=BASE_IMAGE)


@kfp.dsl.pipeline(
    name='Amyris Classifier Training',
    description='The pipeline training and deploying the Amyris classifierpipeline_yaml'
)
def amyris_train(project_id,
                    region,
                    gcs_root,
                    evaluation_metric_name,
                    evaluation_metric_threshold,
                    model_id,
                    version_id,
                    replace_existing_version,
                    hypertune_settings=HYPERTUNE_SETTINGS,
                    dataset_location='US'):
    """Orchestrates training and deployment of an sklearn model."""

    # Create the training split
#     query = generate_sampling_query(
#         source_table_name=source_table_name, num_lots=10, lots=[1, 2, 3, 4])

#     training_file_path = '{}/{}'.format(gcs_root, TRAINING_FILE_PATH)

#     create_training_split = bigquery_query_op(
#         query=query,
#         project_id=project_id,
#         dataset_id=dataset_id,
#         table_id='',
#         output_gcs_path=training_file_path,
#         dataset_location=dataset_location)

#     # Create the validation split
#     query = generate_sampling_query(
#         source_table_name=source_table_name, num_lots=10, lots=[8])

#     validation_file_path = '{}/{}'.format(gcs_root, VALIDATION_FILE_PATH)

#     create_validation_split = bigquery_query_op(
#         query=query,
#         project_id=project_id,
#         dataset_id=dataset_id,
#         table_id='',
#         output_gcs_path=validation_file_path,
#         dataset_location=dataset_location)

    # Create the testing split
#     query = generate_sampling_query(
#         source_table_name=source_table_name, num_lots=10, lots=[9])

#     testing_file_path = '{}/{}'.format(gcs_root, TESTING_FILE_PATH)

#     create_testing_split = bigquery_query_op(
#         query=query,
#         project_id=project_id,
#         dataset_id=dataset_id,
#         table_id='',
#         output_gcs_path=testing_file_path,
#         dataset_location=dataset_location)

    # Tune hyperparameters
    tune_args = [
        #'--training_dataset_path',
       # TRAINING_FILE_PATH,
        '--training_dataset', TRAINING_FILE_DIR,
        '--validation_dataset', VALIDATION_FILE_DIR,
        '--testing_dataset', TESTING_FILE_DIR, 
        '--input_file', INPUT_FILE, 
        '--hptune', 'True'
    ]

    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir/hypertune',
                                kfp.dsl.RUN_ID_PLACEHOLDER)

    hypertune = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri='gcr.io/etl-project-datahub/trainer_image:latest',
        job_dir=job_dir,
        args=tune_args,
        training_input=hypertune_settings)

    # Retrieve the best trial
    get_best_trial = retrieve_best_run_op(
            project_id, hypertune.outputs['job_id'])

    # Train the model on a combined training and validation datasets
    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir', kfp.dsl.RUN_ID_PLACEHOLDER)

    train_args = [
       # '--training_dataset_path',
       #TRAINING_FILE_PATH,
        '--training_dataset', TRAINING_FILE_DIR,
        '--validation_dataset', VALIDATION_FILE_DIR,
        '--testing_dataset',  TESTING_FILE_DIR, 
        '--input_file', INPUT_FILE, 
        '--n_estimators',get_best_trial.outputs['n_estimators'], 
        '--max_leaf_nodes',get_best_trial.outputs['max_leaf_nodes'], 
        '--max_depth',get_best_trial.outputs['max_depth'],
        '--min_samples_split',get_best_trial.outputs['min_samples_split'],
        '--bootstrap' ,get_best_trial.outputs['bootstrap'],
        '--random_state' ,get_best_trial.outputs['random_state'],
        '--max_features' ,get_best_trial.outputs['max_features'],
        '--class_weight' ,get_best_trial.outputs['class_weight'],
        '--min_samples_leaf',get_best_trial.outputs['min_samples_leaf'],
        '--hptune', 'False'
    ]

    train_model = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri='gcr.io/etl-project-datahub/trainer_image:latest',
        job_dir=job_dir,
        args=train_args)

    # Evaluate the model on the testing split
    eval_model = evaluate_model_op(
        dataset_path=TESTING_FILE_PATH,
        model_path=str(train_model.outputs['job_dir']),
        metric_name=evaluation_metric_name)

    # Deploy the model if the primary metric is better than threshold
    with kfp.dsl.Condition(eval_model.outputs['metric_value'] > evaluation_metric_threshold):
        deploy_model = mlengine_deploy_op(
        model_uri=train_model.outputs['job_dir'],
        project_id=project_id,
        model_id=model_id,
        version_id=version_id,
        runtime_version=RUNTIME_VERSION,
        python_version=PYTHON_VERSION,
        replace_existing_version=replace_existing_version)

    # Configure the pipeline to run using the service account defined
      # in the user-gcp-sa k8s secret
    if USE_KFP_SA == 'True':
        kfp.dsl.get_pipeline_conf().add_op_transformer(
              use_gcp_secret('user-gcp-sa'))

Overwriting ./pipeline/amyris_pipeline.py


In [33]:
%%writefile ./pipeline/helper_components.py

# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
"""Helper components."""

from typing import NamedTuple


def retrieve_best_run(project_id: str, job_id: str) -> NamedTuple('Outputs', [('metric_value', float), ('n_estimators', int),
                            ('max_leaf_nodes', int), ('max_depth', int), ('min_samples_split', int), ('min_samples_leaf', int),
                            ('bootstrap', str), ('random_state', int), ('max_features', float), ('class_weight', str)]):
    
    
    """Retrieves the parameters of the best Hypertune run."""

    from googleapiclient import discovery
    from googleapiclient import errors
    
    ml = discovery.build('ml', 'v1')

    job_name = 'projects/{}/jobs/{}'.format(project_id, job_id)
    request = ml.projects().jobs().get(name=job_name)

    try:
        response = request.execute()
    except errors.HttpError as err:
        print(err)
    except:
        print('Unexpected error')

    print(response)

    best_trial = response['trainingOutput']['trials'][0]

    metric_value = best_trial['finalMetric']['objectiveValue']

    n_estimators = int(best_trial['hyperparameters']['n_estimators'])
    max_leaf_nodes = int(best_trial['hyperparameters']['max_leaf_nodes'])
    max_depth = int(best_trial['hyperparameters']['max_depth'])
    min_samples_split = int(best_trial['hyperparameters']['min_samples_split'])
    min_samples_leaf = int(best_trial['hyperparameters']['min_samples_leaf'])
    bootstrap = str(best_trial['hyperparameters']['bootstrap'])
    random_state = int(best_trial['hyperparameters']['random_state'])
    max_features = float(best_trial['hyperparameters']['max_features'])
    class_weight = str(best_trial['hyperparameters']['class_weight'])
        
    return (metric_value, n_estimators, max_leaf_nodes, max_depth, min_samples_split, min_samples_leaf, bootstrap, random_state,max_features, class_weight )


def evaluate_model(dataset_path: str, model_path: str, metric_name: str) -> NamedTuple('Outputs', [('metric_name', str), ('metric_value', float),
                            ('mlpipeline_metrics', 'Metrics')]):
    
    """Evaluates a trained sklearn model."""
    import pickle
    import json
    import pandas as pd
    import subprocess
    import sys

    from sklearn.metrics import accuracy_score, recall_score

    df_test = pd.read_csv(dataset_path)

    X_test = df_test.drop('Run_Performance', axis=1)
    y_test = df_test['Run_Performance']

    # Copy the model from GCS
    model_filename = 'model.pkl'
    gcs_model_filepath = '{}/{}'.format(model_path, model_filename)
    print(gcs_model_filepath)
    subprocess.check_call(['gsutil', 'cp', gcs_model_filepath, model_filename],
                        stderr=sys.stdout)

    with open(model_filename, 'rb') as model_file:
        model = pickle.load(model_file)
        
    y_hat = model.predict(X_test)

    if metric_name == 'accuracy':
        metric_value = accuracy_score(y_test, y_hat)
    elif metric_name == 'recall':
        metric_value = recall_score(y_test, y_hat)
    else:
        metric_name = 'N/A'
        metric_value = 0

    # Export the metric
    metrics = {
      'metrics': [{
          'name': metric_name,
          'numberValue': float(metric_value)
      }]
    }

    return (metric_name, metric_value, json.dumps(metrics))

Overwriting ./pipeline/helper_components.py


In [34]:
TRAINER_IMAGE

'gcr.io/etl-project-datahub/trainer_image:latest'

In [35]:
!cat base_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire scikit-learn==0.20.4 pandas==0.24.2 kfp==0.2.5


In [36]:
!cat train_image/Dockerfile


FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


## Create an empty folder base_image

In [41]:
BASELINE_FOLDER = 'base_image'
os.makedirs(BASELINE_FOLDER, exist_ok=True)

In [42]:
%%writefile ./base_image/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire scikit-learn==0.20.4 pandas==0.24.2 kfp==0.2.5

Overwriting ./base_image/Dockerfile


In [43]:
IMAGE_NAME='base_image'
TAG='latest'
BASE_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [44]:
!gcloud builds submit --timeout 15m --tag $BASE_IMAGE base_image

Creating temporary tarball archive of 1 file(s) totalling 122 bytes before compression.
Uploading tarball of [base_image] to [gs://etl-project-datahub_cloudbuild/source/1599464691.07469-cde00f50db21421ba23ffe86e6d1e972.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/b4a40d09-5505-46ee-ad92-c8c698872017].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/b4a40d09-5505-46ee-ad92-c8c698872017?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "b4a40d09-5505-46ee-ad92-c8c698872017"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599464691.07469-cde00f50db21421ba23ffe86e6d1e972.tgz#1599464691523695
Copying gs://etl-project-datahub_cloudbuild/source/1599464691.07469-cde00f50db21421ba23ffe86e6d1e972.tgz#1599464691523695...
/ [1 files][  228.0 B/  228.0 B]                                                
Operation completed over 1 

# START HERE

# Building and deploying the pipeline

Update the below constants with the settings reflecting your lab environment.

* ##### REGION - the compute region for AI Platform Training and Prediction
* ##### ARTIFACT_STORE - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the hostedkfp-default- prefix.
* ##### ENDPOINT - set the ENDPOINT constant to the endpoint to your AI Platform Pipelines instance. Then endpoint to the * ##### AI Platform Pipelines instance can be found on the <a href="https://console.cloud.google.com/ai-platform/pipelines/clusters">AI Platform Pipelines</a> page in the Google Cloud Console.

* ##### Open the SETTINGS for your instance

* ##### Use the value of the host variable in the Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD section of the SETTINGS window.

In [142]:
!pip3 install kfp --upgrade --user

Requirement already up-to-date: kfp in /opt/conda/lib/python3.7/site-packages (1.0.0)


In [45]:
REGION = 'us-central1'
ENDPOINT = 'caa07ec40e216a8-dot-us-central2.pipelines.googleusercontent.com'
#'19a5aed0f754a516-dot-us-central2.pipelines.googleusercontent.com'
ARTIFACT_STORE_URI = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

# compile pipeline

In [46]:
USE_KFP_SA = False

COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/'
RUNTIME_VERSION = '1.15'
PYTHON_VERSION = '3.7'

%env USE_KFP_SA={USE_KFP_SA}
%env BASE_IMAGE={BASE_IMAGE}
%env TRAINER_IMAGE={TRAINER_IMAGE}
%env COMPONENT_URL_SEARCH_PREFIX={COMPONENT_URL_SEARCH_PREFIX}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}

env: USE_KFP_SA=False
env: BASE_IMAGE=gcr.io/etl-project-datahub/base_image:latest
env: TRAINER_IMAGE=gcr.io/etl-project-datahub/trainer_image:latest
env: COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7


In [47]:
import kfp
client = kfp.Client(host='caa07ec40e216a8-dot-us-central2.pipelines.googleusercontent.com')

In [144]:
!which dsl-compile

/opt/conda/bin/dsl-compile


In [48]:
!dsl-compile --py pipeline/amyris_pipeline.py --output amyris_pipeline.yaml

In [49]:
!head amyris_pipeline.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: amyris-classifier-training-
  annotations: {pipelines.kubeflow.org/kfp_sdk_version: 1.0.0, pipelines.kubeflow.org/pipeline_compilation_time: '2020-09-07T13:06:28.167611',
    pipelines.kubeflow.org/pipeline_spec: '{"description": "The pipeline training
      and deploying the Amyris classifierpipeline_yaml", "inputs": [{"name": "project_id"},
      {"name": "region"}, {"name": "gcs_root"}, {"name": "evaluation_metric_name"},
      {"name": "evaluation_metric_threshold"}, {"name": "model_id"}, {"name": "version_id"},
      {"name": "replace_existing_version"}, {"default": "\n{\n    \"hyperparameters\":  {\n        \"goal\":


# deploy pipeline package

In [50]:
PIPELINE_NAME='amyris_pipeline_RF8'

!kfp --endpoint $ENDPOINT pipeline upload \
-p $PIPELINE_NAME \
amyris_pipeline.yaml

Pipeline 701161a5-26df-4c7a-81b6-4b6f5fc5ab99 has been submitted

Pipeline Details
------------------
ID           701161a5-26df-4c7a-81b6-4b6f5fc5ab99
Name         amyris_pipeline_RF8
Description
Uploaded at  2020-09-07T13:06:39+00:00
+-----------------------------+-------------------------------------------------------+
| Parameter Name              | Default Value                                         |
+=============================+=======================================================+
| project_id                  |                                                       |
+-----------------------------+-------------------------------------------------------+
| region                      |                                                       |
+-----------------------------+-------------------------------------------------------+
| gcs_root                    |                                                       |
+-----------------------------+-----------------------------

In [51]:
!kfp --endpoint $ENDPOINT pipeline list

+--------------------------------------+-------------------------------------------------+---------------------------+
| Pipeline ID                          | Name                                            | Uploaded at               |
+======================================+=================================================+===========================+
| 701161a5-26df-4c7a-81b6-4b6f5fc5ab99 | amyris_pipeline_RF8                             | 2020-09-07T13:06:39+00:00 |
+--------------------------------------+-------------------------------------------------+---------------------------+
| 65886c7c-0c88-4f9a-983b-f1a561cd2276 | amyris_pipeline_RF7                             | 2020-09-07T01:42:27+00:00 |
+--------------------------------------+-------------------------------------------------+---------------------------+
| 0dd4b8c7-167e-4883-a02c-9ac49b63a6bc | amyris_pipeline_RF6                             | 2020-09-06T23:39:34+00:00 |
+--------------------------------------+--------

In [52]:
PIPELINE_ID='701161a5-26df-4c7a-81b6-4b6f5fc5ab99'
#'65886c7c-0c88-4f9a-983b-f1a561cd2276'
#'0dd4b8c7-167e-4883-a02c-9ac49b63a6bc'
#'76e17c3f-8954-4ff5-8099-c3455eb1a723'
#'d88efe41-074b-4535-ab43-83864d58ff13'
#'9afe138a-6b32-4d3b-ac53-8abb7dad137d'
##'0906289c-a5dd-4292-ac8c-71c8925dac26'

In [53]:
EXPERIMENT_NAME = 'amyrisRF_RF2'
RUN_ID = 'Run_001'
EVALUATION_METRIC = 'accuracy'
EVALUATION_METRIC_THRESHOLD = '0.69'
MODEL_ID = 'amyris_RFClassifiervRFC1' #'amyris_endtoendRF2'
VERSION_ID = 'v01'
REPLACE_EXISTING_VERSION = 'True'

GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)

In [54]:
!kfp --endpoint $ENDPOINT run submit \
-e $EXPERIMENT_NAME \
-r $RUN_ID \
-p $PIPELINE_ID \
project_id=$PROJECT_ID \
gcs_root=$GCS_STAGING_PATH \
region=$REGION \
evaluation_metric_name=$EVALUATION_METRIC \
evaluation_metric_threshold=$EVALUATION_METRIC_THRESHOLD \
model_id=$MODEL_ID \
version_id=$VERSION_ID \
replace_existing_version=$REPLACE_EXISTING_VERSION

Creating experiment amyrisRF_RF2.
Run 60c3bfa6-1180-4b9f-8862-211c20c8f50c is submitted
+--------------------------------------+---------+----------+---------------------------+
| run id                               | name    | status   | created at                |
+======================================+=========+==========+===========================+
| 60c3bfa6-1180-4b9f-8862-211c20c8f50c | Run_001 |          | 2020-09-07T13:07:47+00:00 |
+--------------------------------------+---------+----------+---------------------------+


# CI/CD Part

In [58]:
ENDPOINT = 'caa07ec40e216a8-dot-us-central2.pipelines.googleusercontent.com' ##'<YOUR_ENDPOINT>'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
PROJECT_ID

'etl-project-datahub'

In [59]:
KFP_CLI_FOLDER = 'kfp-cli'
os.makedirs(KFP_CLI_FOLDER, exist_ok=True)

In [60]:
%%writefile {KFP_CLI_FOLDER}/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install kfp==0.2.5
ENTRYPOINT ["/bin/bash"]

Overwriting kfp-cli/Dockerfile


In [61]:
!cat kfp-cli/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install kfp==0.2.5
ENTRYPOINT ["/bin/bash"]


In [62]:
IMAGE_NAME='kfp-cli'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [63]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} kfp-cli

Creating temporary tarball archive of 1 file(s) totalling 103 bytes before compression.
Uploading tarball of [kfp-cli] to [gs://etl-project-datahub_cloudbuild/source/1599485809.305995-e348c57df61f4e7c8d4b0ab6e72bca98.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/2db3bc47-5725-42c2-b984-8e3e877d8700].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/2db3bc47-5725-42c2-b984-8e3e877d8700?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "2db3bc47-5725-42c2-b984-8e3e877d8700"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599485809.305995-e348c57df61f4e7c8d4b0ab6e72bca98.tgz#1599485809773088
Copying gs://etl-project-datahub_cloudbuild/source/1599485809.305995-e348c57df61f4e7c8d4b0ab6e72bca98.tgz#1599485809773088...
/ [1 files][  226.0 B/  226.0 B]                                                
Operation completed over 1 

In [64]:
SUBSTITUTIONS="""
_ENDPOINT={},\
_TRAINER_IMAGE_NAME=trainer_image,\
_BASE_IMAGE_NAME=base_image,\
TAG_NAME=latest,\
_PIPELINE_FOLDER=.,\
_PIPELINE_DSL=amyris_pipeline.py,\
_PIPELINE_PACKAGE=amyris_pipeline.yaml,\
_PIPELINE_NAME=amyris_pipeline_RF8,\
_RUNTIME_VERSION=1.15,\
_PYTHON_VERSION=3.7,\
_USE_KFP_SA=True,\
_COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
""".format(ENDPOINT).strip()

In [67]:
%%writefile cloudbuild.yaml
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Submits a Cloud Build job that builds and deploys
# the pipelines and pipelines components 
#
# Build and deploy a TFX pipeline. This is an interim solution till tfx CLI fully 
# supports automated building and deploying.
# 

steps:
# Build the trainer image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/train_image
  
# Build the base image for lightweight components
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/base_image

# Compile the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    dsl-compile --py $_PIPELINE_DSL --output $_PIPELINE_PACKAGE
  env:
  - 'BASE_IMAGE=gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME'
  - 'TRAINER_IMAGE=gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME'
  - 'RUNTIME_VERSION=$_RUNTIME_VERSION'
  - 'PYTHON_VERSION=$_PYTHON_VERSION'
  - 'COMPONENT_URL_SEARCH_PREFIX=$_COMPONENT_URL_SEARCH_PREFIX'
  - 'USE_KFP_SA=$_USE_KFP_SA'
  dir: $_PIPELINE_FOLDER/pipeline
  
 # Upload the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    kfp --endpoint $_ENDPOINT pipeline upload -p ${_PIPELINE_NAME}_$TAG_NAME $_PIPELINE_PACKAGE
  dir: $_PIPELINE_FOLDER/pipeline


# Push the images to Container Registry 
images: ['gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME']

Overwriting cloudbuild.yaml


In [68]:
!gcloud builds submit . --config cloudbuild.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 94 file(s) totalling 28.3 MiB before compression.
Uploading tarball of [.] to [gs://etl-project-datahub_cloudbuild/source/1599486609.236557-e99c5af839c149b78a99e56f30e178d8.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/etl-project-datahub/builds/3abcbdfe-ff4f-4674-bdde-5c501b7bc12d].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/3abcbdfe-ff4f-4674-bdde-5c501b7bc12d?project=448067079266].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "3abcbdfe-ff4f-4674-bdde-5c501b7bc12d"

FETCHSOURCE
Fetching storage object: gs://etl-project-datahub_cloudbuild/source/1599486609.236557-e99c5af839c149b78a99e56f30e178d8.tgz#1599486611822088
Copying gs://etl-project-datahub_cloudbuild/source/1599486609.236557-e99c5af839c149b78a99e56f30e178d8.tgz#1599486611822088...
/ [1 files][  9.5 MiB/  9.5 MiB]                                                
Operation completed over 1 object